In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your openai API key: ")

In [55]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [65]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)

In [22]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [ ]:
# # Load and chunk contents of the blog
# from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://cs2240.graphics")
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)

# # Index chunks
# _ = vector_store.add_documents(documents=all_splits)

In [66]:
from langchain_unstructured import UnstructuredLoader
file_paths = ["csci_courses.txt"]
loader = UnstructuredLoader(file_paths)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [48]:
# from langchain_community.document_loaders import JSONLoader
# import json
# from pathlib import Path


# path = "csci_courses_with_descriptions.json"

# #data = json.loads(Path(path).read_text())
# loader = JSONLoader(
#          file_path=path,
#          jq_schema='.courses',
#          text_content=False)

# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)

# # Index chunks
# _ = vector_store.add_documents(documents=all_splits)

In [67]:
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [59]:
response = graph.invoke({"question": "Tell me good courses on machine learning?"})
print(response["answer"])

Good courses on machine learning include "Machine Learning," which focuses on statistical methods and ethical considerations, and "Systems for Machine Learning," which explores theoretical foundations and algorithm design. Another course emphasizes deep learning and covers topics such as optimization and generalization. Each of these courses provides a solid foundation in both theory and practical applications of machine learning.


In [68]:
response = graph.invoke({"question": "What is Instructor: K. Fisler teaching?"})
print(response["answer"])

I don't know.


In [38]:
response = graph.invoke({"question": "If i want to pursue a career as a AI Enginner in graphics, is this course good?"})
print(response["answer"])

Yes, this course is a solid choice for pursuing a career as an AI Engineer in graphics. It offers advanced knowledge in key areas like rendering, geometry processing, and optimization, preparing you for both academic and industrial research roles. Additionally, the course emphasizes practical project work, which is essential for a successful career in this field.
